In [ ]:
from pathlib import Path
from datetime import datetime, timedelta

import polars as pl

import stock

In [ ]:
csv_dir = stock.PROJECT_ROOT / "data" / "daily"
csv_list = sorted(csv_dir.glob("*.csv"))

In [ ]:
nikkei_df = stock.kabutan.read_data_csv(csv_list[0], exclude_none=False)
topix_df = stock.kabutan.read_data_csv(csv_list[1], exclude_none=False)
df = stock.kabutan.read_data_csv(csv_dir / "6254.csv", exclude_none=False)

In [ ]:
def calc_relative_strength(target_df: pl.DataFrame, reference_df: pl.DataFrame, start_date: datetime, end_date: datetime):
    target_df = target_df.filter((start_date < pl.col("date")) & (pl.col("date") <= end_date))
    reference_df = reference_df.filter((start_date < pl.col("date")) & (pl.col("date") <= end_date))

    if len(target_df) != len(reference_df):
        return -1
    
    return stock.relative_strength.relative_strength(target_df["close"].to_numpy(), reference_df["close"].to_numpy())

In [ ]:
df.filter((start_date < pl.col("date")) & (pl.col("date") <= end_date))

for i in range(len(nikkei_df)):
    date = nikkei_df["date"][i]
    if len(df.filter(pl.col("date") == date)) == 0:
        print(f"Data not found : {date}")

In [ ]:
cur_date = datetime.today()
start_date  = cur_date - timedelta(days=365)
end_date = cur_date

results = []
for csv_path in csv_list:
    df = stock.kabutan.read_data_csv(csv_path, exclude_none=True)
    results.append([csv_path.stem, calc_relative_strength(df, topix_df, start_date, end_date)])

In [ ]:
valid_res = [res for res in results if res[1] > 0]
sorted_res = sorted(valid_res, key=lambda x: x[1])[::-1]
target_codes = [res[0] for res in sorted_res if res[1] > 105]

In [ ]:
# code = "4687"
# date = datetime.strptime("24/03/28", "%y/%m/%d")
# rate = 2

def divide_stock(code: str, date: datetime, rate: float):
    csv_path = stock.PROJECT_ROOT / "data" / "daily" / f"{code}.csv"
    df = stock.kabutan.read_data_csv(csv_path, exclude_none=False)
    def _get_expression(key: str, date: datetime, rate):
        return pl.when(pl.col("date") < date).then(pl.col(key) / rate).otherwise(pl.col(key))

    target_df = df.with_columns(
        pl.col("date").dt.strftime("%Y/%m/%d"),
        _get_expression("open", date, rate),
        _get_expression("high", date, rate),
        _get_expression("low", date, rate),
        _get_expression("close", date, rate),
        _get_expression("volume", date, 1.0 / rate),
    )
    target_df.write_csv(csv_path)

In [ ]:
divide_stock("2652", datetime(year=2024, month=3, day=23), rate=0.2)

In [ ]:
for row in targets:
    date = datetime.strptime(row[0], "%y/%m/%d")
    code = row[1].split(" ")[1]
    splits = row[3].split(" ")
    rate = float(splits[-1]) / float(splits[1])
    print(date, code, rate, row[3])
    divide_stock(code, date, rate)

In [ ]:
import re

import requests
from bs4 import BeautifulSoup

In [ ]:
target_date = datetime.today()
target_date = datetime(year=2024, month=3, day=28)
res = requests.get(f"https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=corporate&dir=corporate&file=stock_ca_bunkatsu.html")
soup = BeautifulSoup(res.content)

In [ ]:
re_date = re.compile("\d\d/\d\d/\d\d")
main = soup.find("div", {"id": "main"})
if main is not None:
    for tr in main.find_all("tr", {"align": "center"}):
        tds = tr.find_all("td")
        if len(tds) != 4:
            continue
        res = re_date.search(tds[0].text)
        if res is None:
            continue
        date = datetime.strptime(res.group(0), "%y/%m/%d")
        if date == target_date:
            code = tds[1].text.split("\xa0")[1]
            splits = tds[3].text.split("\xa0")
            rate = float(splits[3]) / float(splits[1])
            print(code, rate)
        

In [ ]:
res.content

In [ ]:
https://jp.kabumap.com/servlets/kabumap/Action?SRC=splitSch/base
https://jp.kabumap.com/servlets/kabumap/Action?SRC=chgSch/base